In [92]:
import numpy as np

In [148]:
# Constants
num_images = 1  # Total number of images
max_objects_per_image = 3  # Maximum number of objects per image

# Randomly generate the number of objects in each image
num_objects = np.random.randint(0, max_objects_per_image + 1, size=num_images)

# Synthetic data for each object in each image
targets = []
for img_index in range(num_images):
    for obj_index in range(num_objects[img_index]):
        clsid = np.random.randint(0, 3)  # Assume 3 classes, labeled as 0, 1, 2
        cx = np.random.uniform(0, 1000)  # Assume image width up to 1000 pixels
        cy = np.random.uniform(0, 1000)  # Assume image height up to 1000 pixels
        l = np.random.uniform(20, 200)  # Length between 20 and 200 pixels
        s = np.random.uniform(20, 200)  # Width between 20 and 200 pixels
        theta = np.random.uniform(-np.pi/2, np.pi/2)  # Rotation in radians
        targets.append([img_index, clsid, cx, cy, l, s, theta])

# Convert to a NumPy array for better handling
targets = np.array(targets)

# Example use in a loop, similar to your setup
for si in range(num_images):
    labels = targets[targets[:, 0] == si, 1:7]  # Filter labels for the current image
    nl = labels.shape[0]  # Number of labels
    print(f"Image {si} has {nl} labels: {labels}")


Image 0 has 1 labels: [[ 1.00000000e+00  2.54276046e+02  8.16574239e+02  1.06592507e+02
   1.41107777e+02 -3.08615911e-01]]


In [162]:
import numpy as np

# Random example with multiple predictions
num_pred = 1  # Number of predictions
pred = np.random.rand(num_pred, 7)  # Random values for example
pred[:, :2] *= 1000  # Scale center coordinates assuming image size of 1000x1000
pred[:, 2:4] *= 200  # Random dimensions between 0 and 200
pred[:, 4] = pred[:, 4] * np.pi - np.pi/2  # Orientation between -π/2 and π/2
pred[:, 5] = pred[:, 5]  # Confidence scores, already between 0 and 1
pred[:, 6] = np.random.randint(0, 3, size=num_pred)  # Class IDs for 3 classes

print(pred)


[[ 7.07165595e+02  7.13418651e+02  3.12454991e+00  9.96384576e+01
  -8.62434586e-01  1.25778927e-01  0.00000000e+00]]


In [51]:
import torch
 
a = torch.randn(1, 8)
b = a.numel()
print(a)
print(type(b)) # int
print(b) # 24

In [152]:
targets[:1:6]

array([[ 0.00000000e+00,  1.00000000e+00,  2.54276046e+02,
         8.16574239e+02,  1.06592507e+02,  1.41107777e+02,
        -3.08615911e-01]])

In [153]:
len(targets)

1

In [155]:
def rbox2poly(obboxes):
    """
    Trans rbox format to poly format.
    Args:
        rboxes (array/tensor): (num_gts, [cx cy l s θ]) θ∈[-pi/2, pi/2)

    Returns:
        polys (array/tensor): (num_gts, [x1 y1 x2 y2 x3 y3 x4 y4]) 
    """
    if isinstance(obboxes, torch.Tensor):
        center, w, h, theta = obboxes[:, :2], obboxes[:, 2:3], obboxes[:, 3:4], obboxes[:, 4:5]
        Cos, Sin = torch.cos(theta), torch.sin(theta)

        vector1 = torch.cat(
            (w/2 * Cos, -w/2 * Sin), dim=-1)
        vector2 = torch.cat(
            (-h/2 * Sin, -h/2 * Cos), dim=-1)
        point1 = center + vector1 + vector2
        point2 = center + vector1 - vector2
        point3 = center - vector1 - vector2
        point4 = center - vector1 + vector2
        order = obboxes.shape[:-1]
        return torch.cat(
            (point1, point2, point3, point4), dim=-1).reshape(*order, 8)
    else:
        center, w, h, theta = np.split(obboxes, (2, 3, 4), axis=-1)
        Cos, Sin = np.cos(theta), np.sin(theta)

        vector1 = np.concatenate(
            [w/2 * Cos, -w/2 * Sin], axis=-1)
        vector2 = np.concatenate(
            [-h/2 * Sin, -h/2 * Cos], axis=-1)

        point1 = center + vector1 + vector2
        point2 = center + vector1 - vector2
        point3 = center - vector1 - vector2
        point4 = center - vector1 + vector2
        order = obboxes.shape[:-1]
        return np.concatenate(
            [point1, point2, point3, point4], axis=-1).reshape(*order, 8)

poly = rbox2poly(targets[:, :5])
poly

array([[ 213.78289821, -369.35026862,   34.27292428,  427.24853747,
        -213.78289821,  371.35026862,  -34.27292428, -425.24853747]])

In [166]:
poly_tensor = torch.from_numpy(poly)
targets_tensor = torch.from_numpy(pred)

#pred_poly = torch.cat((poly, targets[-2:]), dim=1) # (n, [poly, conf, cls])
# (x1, y1, x2, y2, x3, y3, x4, y4, cx, cy, w, h, θ, conf, cls)
pred_poly = torch.cat((poly_tensor, targets_tensor), dim=1) # (n, [poly, conf, cls])
pred_poly


tensor([[ 2.1378e+02, -3.6935e+02,  3.4273e+01,  4.2725e+02, -2.1378e+02,
          3.7135e+02, -3.4273e+01, -4.2525e+02,  7.0717e+02,  7.1342e+02,
          3.1245e+00,  9.9638e+01, -8.6243e-01,  1.2578e-01,  0.0000e+00]],
       dtype=torch.float64)

In [167]:
def poly2hbb(polys):
    import numpy as np
    """
    Trans poly format to hbb format
    Args:
        rboxes (array/tensor): (num_gts, poly) 

    Returns:
        hbboxes (array/tensor): (num_gts, [xc yc w h]) 
    """
    assert polys.shape[-1] == 8
    if isinstance(polys, torch.Tensor):
        x = polys[:, 0::2] # (num, 4) 
        y = polys[:, 1::2]
        x_max = torch.amax(x, dim=1) # (num)
        x_min = torch.amin(x, dim=1)
        y_max = torch.amax(y, dim=1)
        y_min = torch.amin(y, dim=1)
        x_ctr, y_ctr = (x_max + x_min) / 2.0, (y_max + y_min) / 2.0 # (num)
        h = y_max - y_min # (num)
        w = x_max - x_min
        x_ctr, y_ctr, w, h = x_ctr.reshape(-1, 1), y_ctr.reshape(-1, 1), w.reshape(-1, 1), h.reshape(-1, 1) # (num, 1)
        hbboxes = torch.cat((x_ctr, y_ctr, w, h), dim=1)
    else:
        x = polys[:, 0::2] # (num, 4) 
        y = polys[:, 1::2]
        x_max = np.amax(x, axis=1) # (num)
        x_min = np.amin(x, axis=1) 
        y_max = np.amax(y, axis=1)
        y_min = np.amin(y, axis=1)
        x_ctr, y_ctr = (x_max + x_min) / 2.0, (y_max + y_min) / 2.0 # (num)
        h = y_max - y_min # (num)
        w = x_max - x_min
        x_ctr, y_ctr, w, h = x_ctr.reshape(-1, 1), y_ctr.reshape(-1, 1), w.reshape(-1, 1), h.reshape(-1, 1) # (num, 1)
        hbboxes = np.concatenate((x_ctr, y_ctr, w, h), axis=1)
    return hbboxes

def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # top left x
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # top left y
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # bottom right x
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # bottom right y
    return y

hbbox = xywh2xyxy(poly2hbb(pred_poly[:, :8]))
hbbox

tensor([[-213.7829, -425.2485,  213.7829,  427.2485]], dtype=torch.float64)

In [168]:
pred_hbb = torch.cat((hbbox, pred_poly[:, -2:]), dim=1)
# [x1, y1, x2, y2, confidence, class]
pred_hbb

tensor([[-2.1378e+02, -4.2525e+02,  2.1378e+02,  4.2725e+02,  1.2578e-01,
          0.0000e+00]], dtype=torch.float64)

In [169]:
pred_polyn = pred_poly.clone() # predn (tensor): (n, [poly, conf, cls])
hbboxn = xywh2xyxy(poly2hbb(pred_polyn[:, :8])) # (n, [x1 y1 x2 y2])
print(hbboxn)
pred_hbbn = torch.cat((hbboxn, pred_polyn[:, -2:]), dim=1)
print(pred_hbbn)

tensor([[-213.7829, -425.2485,  213.7829,  427.2485]], dtype=torch.float64)
tensor([[-2.1378e+02, -4.2525e+02,  2.1378e+02,  4.2725e+02,  1.2578e-01,
          0.0000e+00]], dtype=torch.float64)


In [171]:
def rbox2poly(obboxes):
    """
    Trans rbox format to poly format.
    Args:
        rboxes (array/tensor): (num_gts, [cx cy l s θ]) θ∈[-pi/2, pi/2)

    Returns:
        polys (array/tensor): (num_gts, [x1 y1 x2 y2 x3 y3 x4 y4]) 
    """
    if isinstance(obboxes, torch.Tensor):
        center, w, h, theta = obboxes[:, :2], obboxes[:, 2:3], obboxes[:, 3:4], obboxes[:, 4:5]
        Cos, Sin = torch.cos(theta), torch.sin(theta)

        vector1 = torch.cat(
            (w/2 * Cos, -w/2 * Sin), dim=-1)
        vector2 = torch.cat(
            (-h/2 * Sin, -h/2 * Cos), dim=-1)
        point1 = center + vector1 + vector2
        point2 = center + vector1 - vector2
        point3 = center - vector1 - vector2
        point4 = center - vector1 + vector2
        order = obboxes.shape[:-1]
        return torch.cat(
            (point1, point2, point3, point4), dim=-1).reshape(*order, 8)
    else:
        center, w, h, theta = np.split(obboxes, (2, 3, 4), axis=-1)
        Cos, Sin = np.cos(theta), np.sin(theta)

        vector1 = np.concatenate(
            [w/2 * Cos, -w/2 * Sin], axis=-1)
        vector2 = np.concatenate(
            [-h/2 * Sin, -h/2 * Cos], axis=-1)

        point1 = center + vector1 + vector2
        point2 = center + vector1 - vector2
        point3 = center - vector1 - vector2
        point4 = center - vector1 + vector2
        order = obboxes.shape[:-1]
        return np.concatenate(
            [point1, point2, point3, point4], axis=-1).reshape(*order, 8)

tpoly = rbox2poly(targets[:, 1:6]) 
tpoly

array([[-407.0548478 ,  199.22966572, -379.25125474,   96.32717427,
         409.0548478 ,  309.32242612,  381.25125474,  412.22491757]])

In [172]:
tbox = xywh2xyxy(poly2hbb(tpoly))
tbox

array([[-407.0548478 ,   96.32717427,  409.0548478 ,  412.22491757]])